## Deliverable 2. Create a Customer Travel Destinations Map.

In [22]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [23]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Country,Date
0,0,Castro,-24.7911,-50.0119,80.06,44,17,1.99,few clouds,BR,2022-02-12 19:27:31
1,1,Ushuaia,-54.8000,-68.3000,47.86,53,40,37.98,scattered clouds,AR,2022-02-12 19:27:31
2,2,Matara,5.9485,80.5353,75.33,91,100,4.18,overcast clouds,LK,2022-02-12 19:27:32
3,3,Bredasdorp,-34.5322,20.0403,68.29,76,100,5.37,overcast clouds,ZA,2022-02-12 19:27:32
4,4,Lagoa,39.0500,-27.9833,60.17,88,100,22.66,overcast clouds,PT,2022-02-12 19:27:33


In [29]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 85
What is the maximum temperature you would like for your trip? 90


In [30]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_null_df = city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) & \
                 (city_data_df["Max Temp"]>=min_temp)]
preferred_cities_null_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Country,Date
18,18,Comodoro Rivadavia,-45.8667,-67.5000,85.86,23,0,11.50,clear sky,AR,2022-02-12 19:27:37
44,44,Gracias,14.5833,-88.5833,86.02,40,0,6.08,clear sky,HN,2022-02-12 19:28:47
96,96,Trelew,-43.2490,-65.3051,87.78,12,98,2.42,overcast clouds,AR,2022-02-12 19:30:09
191,191,Sangre Grande,10.5835,-61.1177,86.90,60,53,11.43,broken clouds,TT,2022-02-12 19:33:00
192,192,Cabedelo,-6.9811,-34.8339,86.67,71,66,10.16,broken clouds,BR,2022-02-12 19:33:00


In [31]:
# 4a. Determine if there are any empty rows.
preferred_cities_null_df.count()

City_ID        31
City           31
Lat            31
Lng            31
Max Temp       31
Humidity       31
Cloudiness     31
Wind Speed     31
Description    31
Country        31
Date           31
dtype: int64

In [32]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_null_df.dropna()
preferred_cities_df.count()

City_ID        31
City           31
Lat            31
Lng            31
Max Temp       31
Humidity       31
Cloudiness     31
Wind Speed     31
Description    31
Country        31
Date           31
dtype: int64

In [33]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
18,Comodoro Rivadavia,AR,85.86,clear sky,-45.8667,-67.5000,
44,Gracias,HN,86.02,clear sky,14.5833,-88.5833,
96,Trelew,AR,87.78,overcast clouds,-43.2490,-65.3051,
191,Sangre Grande,TT,86.90,broken clouds,10.5835,-61.1177,
192,Cabedelo,BR,86.67,broken clouds,-6.9811,-34.8339,
238,Tutoia,BR,85.98,few clouds,-2.7619,-42.2744,
248,Touros,BR,86.90,scattered clouds,-5.1989,-35.4608,
253,Tela,HN,86.22,scattered clouds,15.7833,-87.4500,
282,Aquiraz,BR,86.09,broken clouds,-3.9014,-38.3911,
284,Siparia,TT,86.79,broken clouds,10.1333,-61.5000,


In [34]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,i in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = i["Lat"]
    lng = i["Lng"]    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ... skipping")
        

Hotel not found ... skipping


In [39]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City           31
Country        31
Max Temp       31
Description    31
Lat            31
Lng            31
Hotel Name     31
dtype: int64

In [43]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [46]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [47]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))